# 드롭아웃 Dropout

- ```nn.Dropout(p = 0.5, inplace = False)```

![대체 텍스트](https://cdn-images-1.medium.com/max/2400/1*iWQzxhVlvadk6VAJjsgXgg.png)

In [ ]:
# 런타임 유형을 GPU로 바꾸시길 추천드립니다.
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [ ]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [ ]:
mnist_train = dset.MNIST("./", 
                         train = True,
                         download = True,
                         transform = transforms.ToTensor(), 
                         target_transform = None)
mnist_test = dset.MNIST("./",
                        train = False,
                        download = True,
                        transform = transforms.ToTensor(), 
                        target_transform = None)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [ ]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size, 
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size, 
                                          shuffle = False,
                                          num_workers = 2, 
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model

In [ ]:
# 드롭아웃을 중간 중간에 넣어줌으로써, 모델이 오버피팅하는 경우에 이를 어느 정도 극복할 수 있습니다.
# 그러나 오버피팅하지 않는 상태에서 Regularization이나 드롭아웃을 넣으면 오히려 학습이 잘 안됩니다.

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.Dropout2d(0.2),                   # p = 0.2로 드롭아웃 설정
            nn.Conv2d(16, 32, 3, padding = 1),   # 28 x 28
            nn.ReLU(),
            nn.Dropout2d(0.2),                   # p = 0.2로 드롭아웃 설정
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.ReLU(),
            nn.Dropout2d(0.2),                   # p = 0.2로 드롭아웃 설정
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.ReLU(),
            nn.Dropout(0.2),                     # p = 0.2로 드롭아웃 설정
            nn.Linear(100, 10)
        )       
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

cuda:0


## 4. Train 

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)         

tensor(2.3091, device='cuda:0', grad_fn=<NllLossBackward>)


In [ ]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[-0.1798, -0.1303, -0.2639],
          [-0.0384, -0.3189, -0.2898],
          [-0.3225, -0.1377, -0.2187]]],


        [[[-0.2394,  0.1309,  0.2936],
          [ 0.2955, -0.1298,  0.1077],
          [ 0.2670, -0.0528,  0.0377]]],


        [[[-0.0903,  0.2565, -0.0442],
          [ 0.2896, -0.0136,  0.1381],
          [-0.1469,  0.2485,  0.0163]]],


        [[[ 0.2077,  0.3145,  0.2904],
          [ 0.0034, -0.2143, -0.2406],
          [ 0.3181, -0.3169, -0.0260]]],


        [[[-0.2721, -0.0369, -0.3013],
          [-0.0474,  0.0503,  0.0707],
          [ 0.2335, -0.2021,  0.1684]]],


        [[[ 0.1931, -0.0550, -0.2350],
          [-0.3100, -0.1345,  0.1539],
          [ 0.1594, -0.2438, -0.1186]]],


        [[[-0.2763, -0.2401, -0.0621],
          [-0.1016, -0.2298,  0.2376],
          [ 0.1367, -0.2832,  0.2019]]],


        [[[ 0.2274,  0.0148,  0.3088],
          [ 0.2361, -0.1202,  0.1103],
          [ 0.1021, -0.3165, -0.0280]]],


        [

## 5. Test

In [ ]:
correct = 0
total = 0

# 배치 정규화나 드롭아웃은 학습할 때와 테스트 할 때 다르게 동작하기 때문에, 모델을 evaluation 모드로 바꿔서 테스트해야 합니다.
model.eval()

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 9.745593070983887


## 정확도가 왜 이렇게 낮을까?

- 오버피팅(과적합)하지 않는 상태에서 드롭아웃을 사용하면 오히려 학습이 잘 안됩니다.